# Two-Photon Calcium Imaging Example

This notebook demonstrates how to use the ndx-microscopy extension for storing and analyzing two-photon calcium imaging data in NWB format. The example includes setting up a microscope with its components, defining a microscopy rig, creating imaging spaces, and handling ROI segmentation and analysis.

## Import Required Libraries

Import necessary packages from NWB, ndx-microscopy, and ndx-ophys-devices extensions, along with other utility libraries.

In [ ]:
from datetime import datetime
from uuid import uuid4
import matplotlib.pyplot as plt
import numpy as np
from pynwb import NWBFile, NWBHDF5IO

from ndx_microscopy import (
    MicroscopeModel,
    Microscope, 
    MicroscopyChannel,
    MicroscopyRig,
    LineScan,
    PlanarImagingSpace,
    PlanarMicroscopySeries,
    PlanarSegmentation,
    SummaryImage,
    MicroscopyResponseSeries,
    MicroscopyResponseSeriesContainer
)

from ndx_ophys_devices import (
    ExcitationSourceModel,
    PulsedExcitationSource,
    BandOpticalFilterModel,
    BandOpticalFilter,
    DichroicMirrorModel,
    DichroicMirror,
    PhotodetectorModel,
    Photodetector,
    Indicator
)

## Create NWB File

Initialize a new NWB file with basic session information.

In [ ]:
nwbfile = NWBFile(
    session_description='Two-photon calcium imaging session',
    identifier=str(uuid4()),
    session_start_time=datetime.now(),
    lab='Neural Imaging Lab',
    institution='University of Neuroscience',
    experiment_description='GCaMP6f imaging in visual cortex'
)

## Set Up Microscope and Components

Configure the microscope and its various components including:
- Microscope model and instance
- Laser source (Chameleon Ultra II)
- Excitation filter
- Dichroic mirror
- Emission filter
- PMT detector
- GCaMP6f indicator

In [ ]:
# Set up microscope model
microscope_model = MicroscopeModel(
    name='2p-model',
    description='Two-photon microscope model for calcium imaging',
    model_number='2p-001',
    manufacturer='Custom Build'
)
nwbfile.add_device(microscope_model)

# Set up microscope with technique
microscope = Microscope(
    name='2p-scope',
    description='Custom two-photon microscope for calcium imaging',
    serial_number='2p-serial-001',
    model=microscope_model,
    technique='mirror scanning'
)
nwbfile.add_device(microscope)

In [ ]:
# Configure laser model and instance
excitation_source_model = ExcitationSourceModel(
    name="chameleon_model",
    manufacturer="Coherent",
    model_number="Chameleon Ultra II",
    description="Excitation source model for two-photon imaging",
    source_type="laser",
    excitation_mode="two-photon",
    wavelength_range_in_nm=[800.0, 1000.0]
)
nwbfile.add_device(excitation_source_model)

laser = PulsedExcitationSource(
    name="chameleon",
    description="Femtosecond pulsed laser for two-photon excitation",
    serial_number="CU2-SN-123456",
    model=excitation_source_model,
    power_in_W=2.5,  # Average power at sample, typically 1-3W for deep imaging
    peak_power_in_W=100000.0,  # 100kW peak power
    peak_pulse_energy_in_J=1.25e-9,  # 1.25 nJ
    pulse_rate_in_Hz=80.0e6,  # 80MHz typical for Chameleon Ultra II
)
nwbfile.add_device(laser)

In [ ]:
# Configure optical components
excitation_filter_model = BandOpticalFilterModel(
    name="excitation_filter_model",
    filter_type="Bandpass",
    manufacturer="Semrock",
    model_number="FF01-920/80",
    center_wavelength_in_nm=920.0,
    bandwidth_in_nm=80.0
)
nwbfile.add_device(excitation_filter_model)

excitation_filter = BandOpticalFilter(
    name="excitation_filter",
    description="Excitation filter for two-photon imaging",
    serial_number="EF-SN-123456",
    model=excitation_filter_model
)
nwbfile.add_device(excitation_filter)

dichroic_mirror_model = DichroicMirrorModel(
    name="primary_dichroic_model",
    manufacturer="Semrock",
    model_number="FF757-Di01",  # Common dichroic for GCaMP imaging
    cut_on_wavelength_in_nm=757.0,  # Transmits >757nm
    cut_off_wavelength_in_nm=750.0,  # Reflects <750nm
    transmission_band_in_nm=[757.0, 1100.0],  # Transmits NIR excitation light
    reflection_band_in_nm=(400.0, 750.0),  # Reflects emission light (including 510nm GCaMP6f emission)
    angle_of_incidence_in_degrees=45.0  # Standard angle for dichroic mirrors in microscopes
)
nwbfile.add_device(dichroic_mirror_model)

dichroic = DichroicMirror(
    name="primary_dichroic",
    description="Dichroic mirror for two-photon imaging",
    serial_number="DM-SN-123456",
    model=dichroic_mirror_model
)
nwbfile.add_device(dichroic)

emission_filter_model = BandOpticalFilterModel(
    name="emission_filter_model",
    filter_type="Bandpass",
    manufacturer="Semrock",
    model_number="FF01-510/84",
    center_wavelength_in_nm=510.0,
    bandwidth_in_nm=84.0
)
nwbfile.add_device(emission_filter_model)

emission_filter = BandOpticalFilter(
    name="emission_filter",
    description="Emission filter for GCaMP6f",
    serial_number="EF-SN-123456",
    model=emission_filter_model
)
nwbfile.add_device(emission_filter)

photodetector_model = PhotodetectorModel(
    name="pmt_model",
    detector_type="PMT",
    manufacturer="Hamamatsu",
    model_number="R6357",
    gain=1000000.0,  # 10^6 typical PMT gain
    gain_unit="V/A"  # Voltage/Current
)
nwbfile.add_device(photodetector_model)

detector = Photodetector(
    name="pmt",
    description="PMT detector for fluorescence detection",
    serial_number="PMT-SN-123456",
    model=photodetector_model
)
nwbfile.add_device(detector)

In [ ]:
# Create indicator
indicator = Indicator(
    name="gcamp6f",
    label="GCaMP6f",
    description="Calcium indicator for two-photon imaging",
    manufacturer="Addgene",
    injection_brain_region="Visual cortex",
    injection_coordinates_in_mm=[-2.5, 3.2, 0.5],
)

## Configure Microscopy Rig

Set up the microscopy rig, connecting all the previously defined components.

In [ ]:
# Create microscopy rig
microscopy_rig = MicroscopyRig(
    name='2p_rig',
    description='Two-photon microscopy rig for calcium imaging',
    microscope=microscope,
    excitation_source=laser,
    excitation_filter=excitation_filter,
    dichroic_mirror=dichroic,
    photodetector=detector,
    emission_filter=emission_filter
)

# Create microscopy channel
microscopy_channel = MicroscopyChannel(
    name="gcamp_channel",
    description="GCaMP6f channel",
    excitation_wavelength_in_nm=920.0,
    emission_wavelength_in_nm=510.0,
    indicator=indicator,
)

## Define Imaging Space and Create Example Data

Set up the imaging space parameters and create sample imaging data.

In [ ]:
# Create example imaging data
frames = 1000
height = 512
width = 512
data = np.random.rand(frames, height, width)

# Define imaging space
illumination_pattern = LineScan(
    name="line_scanning",
    description="Line scanning two-photon microscopy",
    scan_direction="horizontal",
    line_rate_in_Hz=1000.0,
    dwell_time_in_s=1.0e-6,
)

imaging_space = PlanarImagingSpace(
    name="cortex_plane1",
    description="Layer 2/3 of visual cortex",
    pixel_size_in_um=[1.0, 1.0],
    dimensions_in_pixels=[height, width],
    origin_coordinates=[-1.2, -0.6, -2.0],
    location="Visual cortex, layer 2/3",
    reference_frame="bregma",
    orientation="RAS",  # Right-Anterior-Superior
    illumination_pattern=illumination_pattern,
)

# Create imaging series
imaging_series = PlanarMicroscopySeries(
    name='imaging_data',
    description='Two-photon calcium imaging',
    microscopy_channel=microscopy_channel,
    microscopy_rig=microscopy_rig,
    planar_imaging_space=imaging_space,
    data=data,
    unit='a.u.',
    rate=30.0,
    starting_time=0.0
)
nwbfile.add_acquisition(imaging_series)

## ROI Segmentation

Create regions of interest (ROIs) in the imaging data:

In [ ]:
# Create ophys processing module
ophys_module = nwbfile.create_processing_module(
    name='ophys',
    description='Optical physiology processing module'
)

# Create summary images
mean_image = SummaryImage(
    name='mean',
    description='Mean intensity projection',
    data=np.mean(data, axis=0)
)

max_image = SummaryImage(
    name='max',
    description='Maximum intensity projection',
    data=np.max(data, axis=0)
)

# Create segmentation
segmentation = PlanarSegmentation(
    name='rois',
    description='Manual ROI segmentation',
    planar_imaging_space=imaging_space,
    summary_images=[mean_image, max_image]
)

In [ ]:
# Add ROIs using image masks
roi_mask = np.zeros((height, width), dtype=bool)
roi_mask[256:266, 256:266] = True  # 10x10 ROI
segmentation.add_roi(image_mask=roi_mask)

# Create ROI responses
roi_region = segmentation.create_roi_table_region(
    description='All ROIs',
    region=list(range(len(segmentation.id)))
)

# Extract responses (example calculation)
num_rois = len(segmentation.id)
responses = np.zeros((frames, num_rois))

for i, roi_mask in enumerate(segmentation.image_mask[:]):
    roi_data = data[:, roi_mask]
    responses[:, i] = np.mean(roi_data, axis=1)

# Create response series
response_series = MicroscopyResponseSeries(
    name="roi_responses",
    description="Fluorescence responses from ROIs",
    data=responses,
    rois=roi_region,
    unit="n.a.",
    rate=30.0,
    starting_time=0.0,
    microscopy_series=imaging_series,
)

# Create container for response series
response_container = MicroscopyResponseSeriesContainer(
    name='responses',
    microscopy_response_series=[response_series]
)

# Add segmentation and responses to ophys module
ophys_module.add(segmentation)
ophys_module.add(response_container)

## Save and Load NWB File

Write the NWB file to disk and demonstrate how to read it back.

In [ ]:
# Save file
with NWBHDF5IO('calcium_imaging.nwb', 'w') as io:
    io.write(nwbfile)

# Read file and access data
with NWBHDF5IO('calcium_imaging.nwb', 'r') as io:
    nwbfile = io.read()

    # Access imaging data
    imaging = nwbfile.acquisition['imaging_data']
    raw_data = imaging.data[:]

    # Access ROI data
    ophys = nwbfile.processing['ophys']
    rois = ophys['rois']
    roi_masks = rois.image_mask[:]

    # Access responses
    responses = ophys['responses']
    roi_data = responses['roi_responses'].data[:]

## Data Visualization

Create plots to visualize the imaging data, ROI masks, and fluorescence responses.

In [ ]:
# Create a figure with 2 rows and 2 columns
fig = plt.figure(figsize=(20, 15))

# First frame
ax1 = plt.subplot(2, 2, 1)
im1 = ax1.imshow(raw_data[0], cmap="gray")
ax1.set_title("First Frame")
plt.colorbar(im1, ax=ax1)

# ROI masks
ax2 = plt.subplot(2, 2, 2)
for i, mask in enumerate(roi_masks):
    im2 = ax2.imshow(mask, alpha=0.5, cmap="viridis")
    ax2.set_title(f"ROI {i+1} Mask")
plt.colorbar(im2, ax=ax2)

# ROI responses over time
ax3 = plt.subplot(2, 1, 2)
ax3.plot(roi_data)
ax3.set_title('ROI Fluorescence Response')
ax3.set_xlabel('Frame Number')
ax3.set_ylabel('Fluorescence (a.u.)')
ax3.grid(True)

plt.tight_layout()
plt.show()